# LTD Evalautions
This notebook will demponstrate the use of the REST API to evaluate the LTD model.

In [1]:
import requests
from requests.auth import AuthBase
import os
import pandas as pd

We need to grab a token to use the system.

In [2]:
URL ="http://score-web-1:8000"
URL1 = URL+"/api-token-auth/"
payload = {'username':'locomotives', 'password':'locomotives'}
t = requests.post(URL1, data=payload )
token = t.json().get('token')

A small helper to add the token to the headers that get sent to our requests.

In [3]:
class TokenAuth(AuthBase):
    """ Implements a custom authentication scheme. """

    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        """ Attach an API token to a custom auth header. """
        r.headers['Authorization'] = "Token " + f'{self.token}'
        return r

In [4]:
URL2 = URL+"/api/route/list/"
URL3 = URL+"/api/get_consist_list/"
URL4 = URL+"/api/evaluate/"
URL5 = URL+"/api/get_ltd_result/"

In [5]:
r = requests.get(URL2, auth=TokenAuth(token))
routes = r.json()
routes

{'4': 'Tyrone - State College',
 '1': 'Landers Yard - Rickenbacker',
 '2': 'Norfolk - Suffolk',
 '3': 'Savannah - Macon',
 '7': 'Maple Heights Intermodal - Pitcairn Intermodal',
 '5': '47th Street Intermodal - Croxton',
 '8': 'Pitcairn Intermodal - Rutherford Intermodal',
 '9': 'Rutherford Intermodal - Bethlehem Intermodal',
 '11': 'Croxton - 47th Street Intermodal',
 '6': '47th Street Intermodal - Maple Heights Intermodal',
 '12': 'Maple Heights Intermodal - 47th Street Intermodal',
 '13': 'Pitcairn Intermodal - Maple Heights Intermodal',
 '14': 'Rutherford Intermodal - Pitcairn Intermodal',
 '15': 'Bethlehem Intermodal - Rutherford Intermodal',
 '10': 'Bethlehem Intermodal - Croxton',
 '16': 'Croxton - Bethlehem Intermodal'}

In [6]:
r = requests.get(URL3, auth=TokenAuth(token))
consists = r.json()
consists

{'17': '1 diesel : 1 future battery : 50 tank cars',
 '16': '2 diesel :  1 future battery : 50 tank cars',
 '15': '1 diesel : 2 battery : 50 tank cars',
 '14': '1 diesel : 1 battery : 50 tank cars',
 '13': '2 diesel : 1 battery : 50 tank cars',
 '12': '3 diesel : 50 tank cars',
 '11': '2 diesel : 50 tank cars',
 '52': 'fuel cell + future battery + 50 tank cars',
 '53': '3 diesel : 80 double loaded',
 '54': '3 diesel : 80 double empty',
 '55': '3 diesel : 1 future battery : 80 double loaded',
 '56': '3 diesel : 1 future battery : 80 double empty',
 '57': '2d+1b+80 double loaded',
 '58': '2d + 1b + 80 double empty',
 '59': '3 fc + 1 b + 80 double loaded'}

To evaluate the consist on a route, we also need a policy. The policies are currently not stored as an object in the database.
    

In [6]:
policy=["score_lp","hybrid_lp","user_fixed"]

In [7]:
power_order=["diesel","battery","fuelcell"]

In [8]:
braking="maximum_braking"

In [9]:
rapid=True

In [10]:
max_speed=60

In [109]:
data={'routes':12, 'consists': 55, 'policy_type':policy[2], 'power_order': power_order, 'rapid': rapid, 'max_speed': max_speed}

In [110]:
t=requests.post(URL4, data=data, auth=TokenAuth(token))

In [111]:
result = t.json()
result

{'result_id': 565}

In [114]:
r = requests.get(URL5 + str(result['result_id']), auth=TokenAuth(token))
output = r.json()

In [115]:
d2 = output['data']
d3 = {
    'times': d2['times'][1:],
    'distances': d2['distances'][1:],
    'speeds': d2['speeds'][1:],
    'power_total': d2['power']['total'],
    'power_diesel': d2['power']['diesel'],
    'power_battery': d2['power']['battery'],
    'power_regen': d2['power']['regen'],
    'power_track': d2['power']['track'],
    'power_train': d2['power']['train'],
    'power_lost': d2['power']['lost'],
    'power_accel': d2['power']['accel']
}
    

In [116]:
len(d3['speeds'])

7894

In [117]:
df = pd.DataFrame(d3)

In [118]:
df.to_csv('mph-c47-3d-1b-80f-60.csv', index=False)